In [ ]:
import os
from du_astro_utils import calibration, photometry, utils
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.table import Table, vstack
from astropy.stats import sigma_clipped_stats
from scipy.ndimage import median_filter
from astropy.time import Time
from astropy.wcs import WCS
from astropy.wcs.utils import skycoord_to_pixel
from astropy.coordinates import Angle, SkyCoord
import astropy.units as u

## Load galaxies catalogue

In [ ]:
data_dir = os.path.join(utils.C2PU_RES_DIR, utils.DIR_PHOTOM, utils.DIR_GALCLUST)

In [ ]:
reduced = False
aligned = False
for ddir in os.listdir(data_dir):
    subdata_dir = os.path.join(data_dir, ddir)
    if reduced:
        subdata_dir = os.path.join(subdata_dir, "REDUCED")
    if aligned:
        subdata_dir = os.path.join(subdata_dir, "aligned")
    if os.path.isdir(subdata_dir):
        list_fits = [im for im in sorted(os.listdir(subdata_dir)) if ".fits" in im]
        list_fits = sorted(list_fits)
        print(subdata_dir, len(list_fits))

In [ ]:
dirchoice = os.listdir(data_dir)[0]

In [ ]:
gal_cat = Table.read(f"galaxies_{dirchoice}_G_R_RED.fits", format="fits")

In [ ]:
gal_cat

In [ ]:
plt.scatter(gal_cat["AB_MAG_G"] - gal_cat["AB_MAG_R"], gal_cat["AB_MAG_G"])

In [ ]:
plt.hist(gal_cat["AB_MAG_G"] - gal_cat["AB_MAG_R"])

## Answer questions

In [ ]:
gal_df = gal_cat.to_pandas()

In [ ]:
gal_df

In [ ]:
with fits.open(f"coadd_{dirchoice}_SDSSgp_RED.fits") as stack_gp:
    hdr_stack = stack_gp[0].header
    data = stack_gp[0].data
    wcs = WCS(header=hdr_stack)

In [ ]:
hdr_stack.keys

In [ ]:
from astropy.coordinates.name_resolve import NameResolveError

target = hdr_stack.get("OBJECT")
try:
    target_coords = SkyCoord.from_name(target)
except NameResolveError:
    target = (" ".join(target.split("-"))).upper()
    print(target)
    target_coords = SkyCoord.from_name(target)
target_coords

In [ ]:
target_x, target_y = skycoord_to_pixel(target_coords, wcs=wcs)

In [ ]:
mean, med, sigma = sigma_clipped_stats(data, sigma=3)
plt.imshow(data, cmap="gray", vmin=med - 3 * sigma, vmax=med + 5 * sigma)
plt.colorbar()
plt.scatter(gal_cat["xcentroid"], gal_cat["ycentroid"], color="y", alpha=0.6)
plt.scatter(target_x, target_y, color="r", alpha=0.6, s=64)
plt.scatter(data.shape[1] // 2, data.shape[0] // 2, color="g", alpha=0.6)

In [ ]:
clus_centr_coords = SkyCoord.from_pixel(data.shape[1] // 2, data.shape[0] // 2, wcs=wcs)
clus_centr_coords

In [ ]:
gal_df.sort_values("AB_MAG_G", ascending=True, inplace=True)

In [ ]:
gal_df

In [ ]:
m3 = gal_df["AB_MAG_G"][2]
sel_rich = np.logical_and(gal_df["AB_MAG_G"] >= m3, gal_df["AB_MAG_G"] <= m3 + 2)
richness = len(gal_df[sel_rich]) / len(gal_df)

In [ ]:
richness

In [ ]:
gals_coords = SkyCoord(ra=gal_df["ALPHA_J2000"], dec=gal_df["DELTA_J2000"], unit=u.deg)

In [ ]:
dist_to_target = gals_coords.separation(target_coords)

In [ ]:
plt.hist(dist_to_target)